# Azure AI Agent Service Diagnostic Demo


### Import Necessary Libraries
In this cell, we import all the libraries and modules required for the project.
This includes Azure AI SDKs, Gradio for UI, and custom functions.

In [ ]:
import os
import re
import uuid
from datetime import datetime as pydatetime
from typing import Any, List, Dict
from dotenv import load_dotenv

# (Optional) Gradio app for UI
import gradio as gr
from gradio import ChatMessage
import base64

# Azure AI Projects
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.projects import AIProjectClient
import azure.ai.agents as agentslib
import azure.ai.projects as projectslib
from azure.ai.agents.models import (
    AgentEventHandler,
    RunStep,
    RunStepDeltaChunk,
    ThreadMessage,
    ThreadRun,
    MessageDeltaChunk,
    BingGroundingTool,
    FilePurpose,
    FileSearchTool,
    FunctionTool,
    ToolSet,
    VectorStore,
    AzureAISearchTool,
    CodeInterpreterTool,
    MessageDeltaTextContent,
    MessageDeltaImageFileContent,
    MessageDeltaTextContentObject,
    MessageDeltaTextUrlCitationAnnotation,
    MessageRole,
    AgentThread,
    MessageTextContent,
    AgentsNamedToolChoice,
    AgentsToolChoiceOptionMode,
    AgentsNamedToolChoiceType,
)

# Your custom Python functions (for "fetch_datetime", etc.)
from utils.enterprise_functions import enterprise_fns

load_dotenv(dotenv_path=".env", override=True)

from utils.fdyauth import AuthHelper
settings = AuthHelper.load_settings()
credential = AuthHelper.test_credential()

if credential:
    print('Environment and authentication OK')
else:
    print("please login first")

Environment and authentication OK


### Create Client and Load Azure AI Foundry
Here, we initialize the Azure AI client using DefaultAzureCredential.
This allows us to authenticate and connect to the Azure AI service.

In [ ]:
# new AI Foundry Project resource endpoint / old azure ai services endpoint from the hub/project
project_client = AIProjectClient(
    credential=credential,
    endpoint=settings.project_endpoint,
    # api_version=os.environ["PROJECT_API_VERSION"]
)
print("project_client api version:", project_client._config.api_version)
print(f"azure-ai-agents version: {agentslib.__version__}")
print(f"azure-ai-projects version: {projectslib.__version__}")

project_client api version: 2025-05-15-preview
azure-ai-agents version: 1.1.0b3
azure-ai-projects version: 1.0.0b12


### Set Up Tools (BingGroundingTool, FileSearchTool)
In this step, we configure tools such as `BingGroundingTool` and `FileSearchTool`.
We check for existing connections and create or reuse vector stores for document search.

Note:
If you see the following cell has error:
```
AzureCliCredential: Please run 'az login' to set up an account
```

relogin from powershell
```powershell
az logout
az account clear
az login --tenant 00000000-0000-0000-0000-000000000000
```


In [3]:
# try:
#     bing_connection = project_client.connections.get(name=os.environ["BING_CONNECTION_NAME"])
#     # print(f"{bing_connection}")
#     conn_id = bing_connection.id
#     bing_tool = BingGroundingTool(connection_id=conn_id)
#     print("bing > connected")
# except Exception:
#     bing_tool = None
#     print("bing failed > no connection found or permission issue")

## need to be wrapped inside the agents_client, close agents_client if done
FOLDER_NAME = "treatment-data"
VECTOR_STORE_NAME = "lung-treatment-vector-store"

# project_client.agents return the AgentsClient
all_vector_stores: List[VectorStore] = project_client.agents.vector_stores.list()

existing_vector_store = next(
    (store for store in all_vector_stores if store.name == VECTOR_STORE_NAME),
    None
)

vector_store_id = None
if existing_vector_store:
    vector_store_id = existing_vector_store.id
    print(f"reusing vector store > {existing_vector_store.name} (id: {existing_vector_store.id})")
else:
    # If you have local docs to upload
    import os
    if os.path.isdir(FOLDER_NAME):
        file_ids = []
        for file_name in os.listdir(FOLDER_NAME):
            file_path = os.path.join(FOLDER_NAME, file_name)
            if os.path.isfile(file_path):
                print(f"uploading > {file_name}")
                uploaded_file = project_client.agents.files.upload_and_poll(
                    file_path=file_path,
                    purpose=FilePurpose.AGENTS
                )
                file_ids.append(uploaded_file.id)

        if file_ids:
            print(f"creating vector store > from {len(file_ids)} files.")
            vector_store = project_client.agents.vector_stores.create_and_poll(
                file_ids=file_ids,
                name=VECTOR_STORE_NAME
            )
            vector_store_id = vector_store.id
            print(f"created > {vector_store.name} (id: {vector_store_id})")

file_search_tool = None
if vector_store_id:
    file_search_tool = FileSearchTool(vector_store_ids=[vector_store_id])

reusing vector store > lung-treatment-vector-store (id: vs_kvDdrd5AJnCrmuRFP4fJkJWp)


### Set Up Tools (Coding Interpreter)
In this step, we configure tools such as `CodeInterpreterTool`.

In [4]:
# Create Code Interpreter
DATA_FOLDER_NAME = "medicare-data"
uploaded_files_list = project_client.agents.files.list(purpose=FilePurpose.AGENTS).data

# dictionary of uploaded files list with key file.filename, value file.id
uploaded_files_dict = {file.filename: file.id for file in uploaded_files_list}

data_file_ids = []
if os.path.isdir(DATA_FOLDER_NAME):    
    for file_name in os.listdir(DATA_FOLDER_NAME):
        file_path = os.path.join(DATA_FOLDER_NAME, file_name)
        if os.path.isfile(file_path):
            # test file_name exists as key in dictionary uploaded_files_dict
            if file_name in uploaded_files_dict:
                data_file_id = uploaded_files_dict[file_name]
                data_file_ids.append(data_file_id)
                print(f"file already uploaded > {file_name} (id: {data_file_id})")
            else:
                print(f"uploading > {file_name}")
                uploaded_file = project_client.agents.files.upload_and_poll(
                    file_path=file_path,
                    purpose=FilePurpose.AGENTS
                )
                data_file_ids.append(uploaded_file.id)

if len(data_file_ids) > 0:
    code_interpreter_tool = CodeInterpreterTool(file_ids=data_file_ids)
else:
    code_interpreter_tool = None
    print("no data files found to upload")

file already uploaded > lung_cancer_patients_symptoms_2025.csv (id: assistant-GJ6hZx35ajvkaqjsjzrbTB)
file already uploaded > lung_cancer_patient_clinical_features.csv (id: assistant-5ri9uw9EoDCqkYrv5iZvep)


### Combine All Tools into a ToolSet
This step creates a custom `ToolSet` that includes all the tools configured earlier.
It also adds a `LoggingToolSet` subclass to log the inputs and outputs of function calls.

In [5]:
class LoggingToolSet(ToolSet):
    def execute_tool_calls(self, tool_calls: List[Any]) -> List[dict]:
        """
        Execute the upstream calls, printing only two lines per function:
        1) The function name + its input arguments
        2) The function name + its output result
        """

        # For each function call, print the input arguments
        for c in tool_calls:
            if hasattr(c, "function") and c.function:
                fn_name = c.function.name
                fn_args = c.function.arguments
                print(f"{fn_name} inputs > {fn_args} (id:{c.id})")

        # Execute the tool calls (superclass logic)
        raw_outputs = super().execute_tool_calls(tool_calls)

        # Print the output of each function call
        for item in raw_outputs:
            print(f"output > {item['output']}")

        return raw_outputs

custom_functions = FunctionTool(enterprise_fns)

toolset = LoggingToolSet()
# if bing_tool:
#     toolset.add(bing_tool)
if file_search_tool:
    toolset.add(file_search_tool)
if code_interpreter_tool:
    toolset.add(code_interpreter_tool)
toolset.add(custom_functions)

for tool in toolset._tools:
    tool_name = tool.__class__.__name__
    print(f"tool > {tool_name}")
    for definition in tool.definitions:
        if hasattr(definition, "function"):
            fn = definition.function
            print(f"{fn.name} > {fn.description}")
        else:
            pass

tool > FileSearchTool
tool > CodeInterpreterTool
tool > FunctionTool


### Create or Reuse the Enterprise Agent
In this step, we create a new enterprise agent or reuse an existing one.
The agent is configured with a model, instructions, and the toolset from the previous step.

Note:
* You will need to delete the previous agent, while recreate

In [6]:
AGENT_NAME = "med-diagnostic-agent"
found_agent = None
all_agents_list = project_client.agents.list_agents()
for a in all_agents_list:
    if a.name == AGENT_NAME:
        found_agent = a
        break

model_name = settings.model_deployment_name

instructions = (
    "You are a helpful medical diagnostic assistant. "
    "You have access to following tools. \n\n"
    "## Tools:\n"
    " * file_search: get informaton about treatment best practices from your company knowledge\n"
    " * code_interpeter: get information for patient data analytics tasks relating to study data in CSV format. Always load all the CSV files with no index column as DataFrame, and then decide how dataset can help answer the questions.\n"
    "\n"
    "## Instructions:\n"
    "You can use the all the tools to answer questions\n"
    "\n"
    "## Guidelines:\n"
    "Provide well-structured and professional answers. "
    "This year is 2025"
    # "## Execute Plan:\n"
    # "You will execute your plan step by step without further instruction, and return the final answer in the end.\n"
)

project_client.agents.enable_auto_function_calls(tools=toolset)
if found_agent:
    agent = project_client.agents.update_agent(
        agent_id=found_agent.id,
        model=model_name,
        instructions=instructions,
        toolset=toolset,
    )
    project_client.agents.enable_auto_function_calls(tools=toolset) 
    print(f"reusing agent > {agent.name} (id: {agent.id})")
else:
    agent = project_client.agents.create_agent(
        model=model_name,
        name=AGENT_NAME,
        instructions=instructions,
        toolset=toolset,
    )
    print(f"creating agent > {agent.name} (id: {agent.id})")

reusing agent > med-diagnostic-agent (id: asst_h0TzV6BRST4V9e3sc33jKYpW)


### Create a Conversation Thread
In this step, we create a new conversation thread for the enterprise agent.
Threads are used to manage and track conversations with the agent.

In [7]:
thread = project_client.agents.threads.create()
print(f"thread > created (id: {thread.id})")

thread > created (id: thread_8g4b0MWnqjSw4z6phgTat9MD)


### Define a Custom Event Handler
Here, we define a custom event handler to manage logs and outputs for debugging.
This handler will capture and display real-time events during the agent's operation.

In [8]:
class MyEventHandler(AgentEventHandler):
    def __init__(self):
        super().__init__()
        self._current_message_id = None
        self._accumulated_text = ""

    def on_message_delta(self, delta: MessageDeltaChunk) -> None:
        # If a new message id, start fresh
        if delta.id != self._current_message_id:
            # First, if we had an old message that wasn't completed, finish that line
            if self._current_message_id is not None:
                print()  # move to a new line
            
            self._current_message_id = delta.id
            self._accumulated_text = ""
            print("\nassistant > ", end="")  # prefix for new message

        # Accumulate partial text
        partial_text = ""
        if delta.delta.content:
            for chunk in delta.delta.content:
                # partial_text += chunk.text.get("value", "")
                if isinstance(chunk, MessageDeltaTextContent):
                    partial_text += chunk["text"].get("value", "")
                elif isinstance(chunk, MessageDeltaImageFileContent):
                    partial_text += chunk["image_file"].get("file_id", "")
        self._accumulated_text += partial_text

        # Print partial text with no newline
        print(partial_text, end="", flush=True)

    def on_thread_message(self, message: ThreadMessage) -> None:
        # When the assistant's entire message is "completed", print a final newline
        if message.status == "completed" and message.role == "assistant":
            print()  # done with this line
            self._current_message_id = None
            self._accumulated_text = ""
        else:
            # For other roles or statuses, you can log if you like:
            print(f"{message.status.name.lower()} (id: {message.id})")

    def on_thread_run(self, run: ThreadRun) -> None:
        print(f"status > {run.status.name.lower()}")
        if run.status == "failed":
            print(f"error > {run.last_error}")

    def on_run_step(self, step: RunStep) -> None:
        print(f"{step.type.name.lower()} > {step.status.name.lower()}")

    def on_run_step_delta(self, delta: RunStepDeltaChunk) -> None:
        # If partial tool calls come in, we log them
        if delta.delta.step_details and delta.delta.step_details.tool_calls:
            for tcall in delta.delta.step_details.tool_calls:
                if getattr(tcall, "function", None):
                    if tcall.function.name is not None:
                        print(f"tool call > {tcall.function.name}")

    def on_unhandled_event(self, event_type: str, event_data):
        print(f"unhandled > {event_type} > {event_data}")

    def on_error(self, data: str) -> None:
        print(f"error > {data}")

    def on_done(self) -> None:
        print("done")

### Implement the Main Chat Functions
These functions define how user messages and tool interactions are processed.
It uses the agent's thread to handle conversations and streams partial responses.

In [9]:
def extract_bing_query(request_url: str) -> str:
    """
    Extract the query string from something like:
      https://api.bing.microsoft.com/v7.0/search?q="latest news about Microsoft January 2025"
    Returns: latest news about Microsoft January 2025
    """
    match = re.search(r'q="([^"]+)"', request_url)
    if match:
        return match.group(1)
    # If no match, fall back to entire request_url
    return request_url

def extract_search_annotation(
        annotation: MessageDeltaTextUrlCitationAnnotation, text_value_str: str) -> str:
    """
    {'index': 0, 'type': 'text', 'text': {'value': '【3:1†Siemens fiscal report 2024】', 'annotations': [{'index': 0, 'type': 'url_citation', 'text': '【3:1†Siemens fiscal report 2024】', 'start_index': 1369, 'end_index': 1401, 'url_citation': {'url': 'doc_1', 'title': 'Siemens_Report_FY2024.pdf'}}]}}

    {'index': 0, 'type': 'text', 'text': {'value': '【14:0†best_practices_lung_cancer.md】', 'annotations': [{'index': 0, 'type': 'file_citation', 'text': '【14:0†best_practices_lung_cancer.md】', 'start_index': 2486, 'end_index': 2522, 'file_citation': {'file_id': 'assistant-XfXFvez3N2CbpttpNb8MK4', 'quote': ''}}]}}
    """
    if annotation["type"] == "url_citation":
        url = annotation["url_citation"]["url"]
        title = annotation["url_citation"].get("title", "")
        start_idx = annotation.get("start_index", 0)
        end_idx = annotation.get("end_index", 0)
        return f" [{text_value_str.strip()} {title}]({url}) ({start_idx}-{end_idx})"
    elif annotation["type"] == "file_citation":
        # file_id = annotation["file_citation"]["file_id"]
        file_id = ""
        quote = annotation["file_citation"].get("quote", "")
        title = annotation["text"]
        start_idx = annotation.get("start_index", 0)
        end_idx = annotation.get("end_index", 0)
        return f" [{text_value_str.strip()} {title}]({file_id}) ({start_idx}-{end_idx})"
    else:
        return ""

def convert_dict_to_chatmessage(msg: dict) -> ChatMessage:
    """
    Convert a legacy dict-based message to a gr.ChatMessage.
    Uses the 'metadata' sub-dict if present.
    """
    return ChatMessage(
        role=msg["role"],
        content=msg["content"],
        metadata=msg.get("metadata", None)
    )

def azure_enterprise_chat(user_message: str, history: List[dict]):
    """
    Accumulates partial function arguments into ChatMessage['content'], sets the
    corresponding tool bubble status from "pending" to "done" on completion,
    and also handles non-function calls like bing_grounding or file_search by appending a
    "pending" bubble. Then it moves them to "done" once tool calls complete.

    This function returns a list of ChatMessage objects directly (no dict conversion).
    Your Gradio Chatbot should be type="messages" to handle them properly.
    """

    # Convert existing history from dict to ChatMessage
    conversation = []
    for msg_dict in history:
        conversation.append(convert_dict_to_chatmessage(msg_dict))

    # Append the user's new message
    conversation.append(ChatMessage(role="user", content=user_message))

    # Immediately yield two outputs to clear the textbox
    yield conversation, ""

    # Post user message to the thread (for your back-end logic)
    project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_message
    )

    # Mappings for partial function calls
    call_id_for_index: Dict[int, str] = {}
    partial_calls_by_index: Dict[int, dict] = {}
    partial_calls_by_id: Dict[str, dict] = {}
    in_progress_tools: Dict[str, ChatMessage] = {}

    # Titles for tool bubbles
    function_titles = {
        # "fetch_weather": "☁️ fetching weather",
        "fetch_datetime": "🕒 fetching datetime",
        # "fetch_stock_price": "📈 fetching financial info",
        # "send_email": "✉️ sending mail",
        "file_search": "📄 searching docs",
        "bing_grounding": "🔍 searching bing",
        "code_interpreter": "📊 analyzing data",
    }

    def get_function_title(fn_name: str) -> str:
        return function_titles.get(fn_name, f"🛠 calling {fn_name}")

    def accumulate_args(storage: dict, name_chunk: str, arg_chunk: str):
        """Accumulates partial JSON data for a function call."""
        if name_chunk:
            storage["name"] += name_chunk
        if arg_chunk:
            storage["args"] += arg_chunk

    def finalize_tool_call(call_id: str):
        """Creates or updates the ChatMessage bubble for a function call."""
        if call_id not in partial_calls_by_id:
            return
        data = partial_calls_by_id[call_id]
        fn_name = data["name"].strip()
        fn_args = data["args"].strip()
        if not fn_name:
            return

        if call_id not in in_progress_tools:
            # Create a new bubble with status="pending"
            msg_obj = ChatMessage(
                role="assistant",
                content=fn_args or "",
                metadata={
                    "title": get_function_title(fn_name),
                    "status": "pending",
                    "id": f"tool-{call_id}"
                }
            )
            conversation.append(msg_obj)
            in_progress_tools[call_id] = msg_obj
        else:
            # Update existing bubble
            msg_obj = in_progress_tools[call_id]
            msg_obj.content = fn_args or ""
            msg_obj.metadata["title"] = get_function_title(fn_name)

    def upsert_tool_call(tcall: dict):
        """
        1) Check the call type
        2) If "function", gather partial name/args
        3) If "bing_grounding" or "file_search", show a pending bubble
        4) If "code_interpreter", create only one bubble
        """
        t_type = tcall.get("type", "")
        call_id = tcall.get("id", None)

        # If call_id is None, generate a unique one (for parallel calls)
        if call_id is None:
            if t_type in ("file_search", "bing_grounding"):
                call_id = str(uuid.uuid4())
            elif t_type == "code_interpreter":
                call_id = "code_interpreter"
            elif t_type == "azure_ai_search":
                call_id = "azure_ai_search"
            else:
                call_id = "unknown_tool"

        # --- BING GROUNDING ---
        if t_type == "bing_grounding":
            request_url = tcall.get("bing_grounding", {}).get("requesturl", "")
            if not request_url.strip():
                return

            query_str = extract_bing_query(request_url)
            if not query_str.strip():
                return

            msg_obj = ChatMessage(
                role="assistant",
                content=query_str,
                metadata={
                    "title": get_function_title("bing_grounding"),
                    "status": "pending",
                    "id": f"tool-{call_id}" if call_id else "tool-noid"
                }
            )
            conversation.append(msg_obj)
            if call_id is not None:
                in_progress_tools[call_id] = msg_obj
            return

        # --- FILE SEARCH ---
        elif t_type == "file_search":
            msg_obj = ChatMessage(
                role="assistant",
                content="searching docs...",
                metadata={
                    "title": get_function_title("file_search"),
                    "status": "pending",
                    "id": f"tool-{call_id}" if call_id else "tool-noid"
                }
            )
            conversation.append(msg_obj)
            if call_id is not None:
                in_progress_tools[call_id] = msg_obj
            return
        
        # --- Azure AI SEARCH --- 
        elif t_type == "azure_ai_search":
            if call_id not in in_progress_tools:
                msg_obj = ChatMessage(
                    role="assistant",
                    content="searching private index...",
                    metadata={
                        "title": get_function_title("azure_ai_search"),
                        "status": "pending",
                        "id": f"tool-{call_id}" if call_id else "tool-noid"
                    }
                )
                conversation.append(msg_obj)
                in_progress_tools[call_id] = msg_obj
            return
        
        # -- CODE INTERPRETER ---
        elif t_type == "code_interpreter":
            if call_id not in in_progress_tools:
                msg_obj = ChatMessage(
                    role="assistant",
                    content="analyzing data...",
                    metadata={
                        "title": get_function_title("code_interpreter"),
                        "status": "pending",
                        "id": f"tool-{call_id}"
                    }
                )
                conversation.append(msg_obj)
                in_progress_tools[call_id] = msg_obj
            return

        # --- NON-FUNCTION CALLS ---
        elif t_type != "function":
            return

        # --- FUNCTION CALL PARTIAL-ARGS ---
        index = tcall.get("index")
        new_call_id = call_id
        fn_data = tcall.get("function", {})
        name_chunk = fn_data.get("name", "")
        arg_chunk = fn_data.get("arguments", "")

        if new_call_id:
            call_id_for_index[index] = new_call_id

        call_id = call_id_for_index.get(index)
        if not call_id:
            # Accumulate partial
            if index not in partial_calls_by_index:
                partial_calls_by_index[index] = {"name": "", "args": ""}
            accumulate_args(partial_calls_by_index[index], name_chunk, arg_chunk)
            return

        if call_id not in partial_calls_by_id:
            partial_calls_by_id[call_id] = {"name": "", "args": ""}

        if index in partial_calls_by_index:
            old_data = partial_calls_by_index.pop(index)
            partial_calls_by_id[call_id]["name"] += old_data.get("name", "")
            partial_calls_by_id[call_id]["args"] += old_data.get("args", "")

        # Accumulate partial
        accumulate_args(partial_calls_by_id[call_id], name_chunk, arg_chunk)

        # Create/update the function bubble
        finalize_tool_call(call_id)

    # -- EVENT STREAMING --
    with project_client.agents.runs.stream(
        thread_id=thread.id,
        agent_id=agent.id,
        # assistant_id=agent.id,
        event_handler=MyEventHandler(),  # the event handler handles console output
        parallel_tool_calls=True,
        tool_choice=AgentsNamedToolChoice(
            type=AgentsNamedToolChoiceType.CODE_INTERPRETER), 
    ) as stream:
        # pulling the result from the stream manually
        for item in stream:
            event_type, event_data, *_ = item

            # Remove any None items that might have been appended
            conversation = [m for m in conversation if m is not None]

            # 1) Partial tool calls
            if event_type == "thread.run.step.delta":
                step_delta = event_data.get("delta", {}).get("step_details", {})
                if step_delta.get("type") == "tool_calls":
                    for tcall in step_delta.get("tool_calls", []):
                        upsert_tool_call(tcall)
                    yield conversation, ""

            # 2) run_step
            elif event_type == "run_step":
                step_type = event_data["type"]
                step_status = event_data["status"]

                # If tool calls are in progress, new or partial
                if step_type == "tool_calls" and step_status == "in_progress":
                    for tcall in event_data["step_details"].get("tool_calls", []):
                        upsert_tool_call(tcall)
                    yield conversation, ""

                elif step_type == "tool_calls" and step_status == "completed":
                    for cid, msg_obj in in_progress_tools.items():
                        msg_obj.metadata["status"] = "done"
                    in_progress_tools.clear()
                    partial_calls_by_id.clear()
                    partial_calls_by_index.clear()
                    call_id_for_index.clear()
                    yield conversation, ""

                elif step_type == "message_creation" and step_status == "in_progress":
                    msg_id = event_data["step_details"]["message_creation"].get("message_id")
                    if msg_id:
                        conversation.append(ChatMessage(role="assistant", content=""))
                    yield conversation, ""

                elif step_type == "message_creation" and step_status == "completed":
                    yield conversation, ""

            # 3) partial text from the assistant
            elif event_type == "thread.message.delta":
                agent_msg = ""
                for chunk in event_data["delta"]["content"]:
                    # print("chunk > ", chunk)
                    # print("chunk type > ", type(chunk))
                    if isinstance(chunk, MessageDeltaTextContent):
                        # Safely get the text value
                        text_obj: MessageDeltaTextContentObject = chunk.get("text", {})
                        text_value_str = text_obj.get("value", "")
                        annotations = text_obj.get("annotations", None)
                        if annotations:
                            # Extract the URL citation if available
                            for annotation in annotations: 
                                agent_msg += extract_search_annotation(annotation, text_value_str)
                        else:
                            agent_msg += text_value_str
                            
                    elif isinstance(chunk, MessageDeltaImageFileContent):
                        file_id = chunk["image_file"].get("file_id", "")
                        byte_stream = project_client.agents.files.get_content(file_id=file_id)
                        # Encode to base64
                        # Join all bytes from the iterator
                        image_bytes = b"".join(byte_stream)
                        b64_image = base64.b64encode(image_bytes).decode("utf-8")
                        # Use Markdown to display the image in Gradio
                        agent_msg += f"![image](data:image/png;base64,{b64_image})"  

                message_id = event_data["id"]

                # Try to find a matching assistant bubble
                matching_msg = None
                for msg in reversed(conversation):
                    if msg.metadata and msg.metadata.get("id") == message_id and msg.role == "assistant":
                        matching_msg = msg
                        break

                if matching_msg:
                    # Append newly streamed text
                    matching_msg.content += agent_msg
                else:
                    # Append to last assistant or create new
                    if (
                        not conversation
                        or conversation[-1].role != "assistant"
                        or (
                            conversation[-1].metadata
                            and str(conversation[-1].metadata.get("id", "")).startswith("tool-")
                        )
                    ):
                        conversation.append(ChatMessage(role="assistant", content=agent_msg))
                    else:
                        conversation[-1].content += agent_msg

                yield conversation, ""

            # 4) If entire assistant message is completed
            elif event_type == "thread.message":
                if event_data["role"] == "assistant" and event_data["status"] == "completed":
                    for cid, msg_obj in in_progress_tools.items():
                        msg_obj.metadata["status"] = "done"
                    in_progress_tools.clear()
                    partial_calls_by_id.clear()
                    partial_calls_by_index.clear()
                    call_id_for_index.clear()
                    yield conversation, ""

            # 5) Final done
            elif event_type == "thread.message.completed":
                for cid, msg_obj in in_progress_tools.items():
                    msg_obj.metadata["status"] = "done"
                in_progress_tools.clear()
                partial_calls_by_id.clear()
                partial_calls_by_index.clear()
                call_id_for_index.clear()
                yield conversation, ""
                break

    return conversation, ""

### Build a Gradio UI
Create a Gradio interface for interacting with the enterprise agent. 
Include a chatbot component and a text input box for user queries.

In [10]:
brand_theme = gr.themes.Default(
    primary_hue="blue",
    secondary_hue="blue",
    neutral_hue="gray",
    font=["Segoe UI", "Arial", "sans-serif"],
    font_mono=["Courier New", "monospace"],
    text_size="lg",
).set(
    button_primary_background_fill="#0f6cbd",
    button_primary_background_fill_hover="#115ea3",
    button_primary_background_fill_hover_dark="#4f52b2",
    button_primary_background_fill_dark="#5b5fc7",
    button_primary_text_color="#ffffff",
    button_secondary_background_fill="#e0e0e0",
    button_secondary_background_fill_hover="#c0c0c0",
    button_secondary_background_fill_hover_dark="#a0a0a0",
    button_secondary_text_color="#000000",
    body_background_fill="#f5f5f5",
    block_background_fill="#ffffff",
    body_text_color="#242424",
    body_text_color_subdued="#616161",
    block_border_color="#d1d1d1",
    block_border_color_dark="#333333",
    input_background_fill="#ffffff",
    input_border_color="#d1d1d1",
    input_border_color_focus="#0f6cbd",
)

with gr.Blocks(theme=brand_theme, css="footer {visibility: hidden;}", fill_height=True) as demo:

    def clear_thread():
        global thread
        thread = project_client.agents.threads.create()
        return []

    def on_example_clicked(evt: gr.SelectData):
        return evt.value["text"]  # Fill the textbox with that example text

    gr.HTML("<h1 style=\"text-align: center;\">Azure AI Agent Service</h1>")
    
    # patient 10
    # Cough and wheezing for 5 months.
    # You need to "continue" if the Agent generate a plan.
    chatbot = gr.Chatbot(
        type="messages",
        examples=[
            # {"text": "Patient 10 has\n1.) The aorta is atherosclerotic.\n2.) A 4.5 cm mass is in the left lower lobe.\n3.) There are chronic healed bilateral rib fractures.\n4.) The lungs are hyperinflated.The heart size is normal."},
            {"text": "Is patient with PATIENT_ID 10 an outlier focusing on symptoms and risk factors from the lung_cancer_patient_clinical_features table, Use LOF and generate a boxplot chart for all normalized features?"},
            {"text": "Suggest a treatment plan with research innovation from best practices for patient with PATIENT_ID 10."},
        ],
        show_label=False,
        scale=1,
    )

    textbox = gr.Textbox(
        show_label=False,
        lines=1,
        submit_btn=True,
    )

    # Populate textbox when an example is clicked
    chatbot.example_select(fn=on_example_clicked, inputs=None, outputs=textbox)

    # On submit: call azure_enterprise_chat, then clear the textbox
    (textbox
     .submit(
         fn=azure_enterprise_chat,
         inputs=[textbox, chatbot],
         outputs=[chatbot, textbox],
     )
     .then(
         fn=lambda: "",
         outputs=textbox,
     )
    )

    # A "Clear" button that resets the thread and the Chatbot
    chatbot.clear(fn=clear_thread, outputs=chatbot)

# Launch your Gradio app
if __name__ == "__main__":
    print("Note:\ntool calling may fail, close the chat with trash bin icon (🗑️) and rerun the prompt.\n")
    demo.launch()

Note:
tool calling may fail, close the chat with trash bin icon (🗑️) and rerun the prompt.

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


### (Optional) delete agent, thread, and vector store resources
Uncomment out the next cell block to delete the resources created in this notebook.

In [11]:
# from azure.identity import DefaultAzureCredential
# from azure.ai.projects import AIProjectClient
# import os

# credential = DefaultAzureCredential()
# project_client_delete = AIProjectClient(
#    credential=credential,
#    endpoint=os.environ.get("PROJECT_ENDPOINT")
# )

# try:
#    project_client_delete.agents.delete_agent(agent.id)
#    print("Agent deletion successful.")
#    project_client_delete.agents.threads.delete(thread.id)
#    print("Thread deletion successful.")
#    project_client_delete.agents.vector_stores.delete(vector_store_id)
#    print("Vector store deletion successful.")
#    print("All deletions succeeded.")
# except Exception as e:
#    print(f"Error during deletion: {e}")

In [ ]:
# existing_stores = project_client_delete.agents.vector_stores.list()
# for store in existing_stores:
#     project_client_delete.agents.vector_stores.delete(store.id)

status > queued
status > queued
status > in_progress
tool_calls > in_progress
tool_calls > in_progress
tool_calls > completed
tool_calls > in_progress
tool_calls > in_progress
tool_calls > completed
tool_calls > in_progress
tool_calls > in_progress
tool_calls > completed
message_creation > in_progress
message_creation > in_progress
in_progress (id: msg_KXEAL73l4k8HcjlWyqFoWd3c)
in_progress (id: msg_KXEAL73l4k8HcjlWyqFoWd3c)

assistant > assistant-GsMFbmZF7Bta3jtkSwRRczThe patient with PATIENT_ID 10 is not classified as an outlier based on the Local Outlier Factor (LOF) analysis on symptoms and risk factors features from the lung_cancer_patient_clinical_features table. The LOF score for this patient is approximately 1.00, which indicates typical behavior rather than being an outlier.

I have also generated a boxplot of all the normalized features used in the analysis, which visually represents the distribution and spread of the standardized clinical features among the patients.

If you 